In [1]:
import os, sys
import pandas as pd

In [2]:
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
from util.feature import get_feat_d_from_ID
from util.genome import is_overlap
from util.params import ENRICH_ALPHA, MULTI_HYP_CORR_METHOD
pd.options.display.max_columns = 100

In [3]:
exp_muts_df = pd.read_pickle("./data/10_df.pkl")
display(exp_muts_df.shape, exp_muts_df.head())

(114, 45)

,index,Details,mutation target annotation,Mutation Type,Position,Reference Seq,Sequence Change,ale,exp,flask,isolate,presence,tech_rep,coding,range,gene RegulonDB ID,genetic features,oriC,pseudogene,TFBS,promoter,RBS,attenuator terminator,terminator,genetic,genomic features,genetic feature links,operons,operon links,regulators,regulator links,pathways,pathway links,COGs,COG links,temperature,carbon-source,supplement,strain-description,taxonomy-id,base-media,nitrogen-source,phosphorous-source,sulfur-source,calcium-source
2527,28,None,[crl],DEL,257908,NC_000913,Δ776 bp,1,TOL_putrescine,50,1,1.0,1,True,"(257908, 258683)","{ECK125229291, ECK120001080, ECK125229290}","[{'name': 'insA9', 'RegulonDB ID': 'ECK1252292...",False,False,{},{},{},{},{},True,"[{'name': 'insB9', 'RegulonDB ID': 'ECK1252292...","{'ECK125229291': ['ECK125229291'], 'ECK1252292...",[],{},[],{},[],{},[],{},37 celsius,glucose(2),putrescine,WT,511145,M9,NH4Cl(1),KH2PO4(3) Na2HPO4(6.8),MgSO4(0.24),CaCl2(0.1)
2528,3,coding (163/210 nt),cspC,INS,1907273,NC_000913,(CGTCCTG)1→2,1,TOL_putrescine,50,1,1.0,1,True,"(1907273, 1907273)",{ECK120002092},"[{'name': 'cspC', 'RegulonDB ID': 'ECK12000209...",False,False,{},{},{},{},{},True,"[{'name': 'cspC', 'RegulonDB ID': 'ECK12000209...",{'ECK120002092': ['ECK120002092']},"[{'name': 'yobF-cspC', 'RegulonDB ID': 'ECK120...",{'ECK120028784': ['ECK120002092']},[],{},[],{},"[{'name': 'Transcription', 'mutation set count...",{'Transcription': ['ECK120028784']},37 celsius,glucose(2),putrescine,WT,511145,M9,NH4Cl(1),KH2PO4(3) Na2HPO4(6.8),MgSO4(0.24),CaCl2(0.1)
2529,29,None,"insB1, insA",DEL,1978503,NC_000913,Δ776 bp,1,TOL_putrescine,50,1,1.0,1,True,"(1978503, 1979278)","{ECK120003617, ECK120003616, ECK120023915}","[{'name': 'insB-5', 'RegulonDB ID': 'ECK120003...",False,False,"{ECK120013472, ECK120017150}","{ECK125095446, ECK120034252}",{},{},{},True,"[{'name': 'flhDC TFBS', 'RegulonDB ID': 'ECK12...","{'ECK120003616': ['ECK120013472', 'ECK12000361...","[{'name': 'flhDC', 'RegulonDB ID': 'ECK1200149...","{'ECK120014934': ['ECK120003616', 'ECK12000361...","[{'name': 'YjjQ', 'significantly associated co...","{'YjjQ': ['flhDC', 'flhDC'], 'Fur': ['flhDC', ...",[],{},"[{'name': 'Mobilome: prophages, transposons', ...","{'Mobilome: prophages, transposons': ['ECK1250...",37 celsius,glucose(2),putrescine,WT,511145,M9,NH4Cl(1),KH2PO4(3) Na2HPO4(6.8),MgSO4(0.24),CaCl2(0.1)
2530,32,intergenic (‑1/+1),gatC/gatC,DEL,2173363,NC_000913,Δ2 bp,1,TOL_putrescine,50,1,1.0,1,False,"(2173363, 2173364)",{ECK120002276},"[{'name': 'gatC', 'RegulonDB ID': 'ECK12000227...",False,False,{},{},{},{},{},False,"[{'name': 'gatC', 'RegulonDB ID': 'ECK12000227...",{'ECK120002276': ['ECK120002276']},"[{'name': 'gatYZABCD', 'RegulonDB ID': 'ECK120...",{'ECK120015849': ['ECK120002276']},"[{'name': 'GatR', 'significantly associated co...","{'GatR': ['gatYZABCD'], 'ArcA': ['gatYZABCD'],...",[],{},[],{},37 celsius,glucose(2),putrescine,WT,511145,M9,NH4Cl(1),KH2PO4(3) Na2HPO4(6.8),MgSO4(0.24),CaCl2(0.1)
2531,4,N34K (AAT→AAG),mreB,SNP,3400986,NC_000913,A→C,1,TOL_putrescine,50,1,1.0,1,True,"(3400986, 3400986)",{ECK120000601},"[{'name': 'mreB', 'RegulonDB ID': 'ECK12000060...",False,False,{},{},{},{},{},True,"[{'name': 'mreB', 'RegulonDB ID': 'ECK12000060...",{'ECK120000601': ['ECK120000601']},"[{'name': 'mreBCD', 'RegulonDB ID': 'ECK120030...",{'ECK120030764': ['ECK120000601']},[],{},[],{},"[{'name': 'Cell cycle control, cell division, ...","{'Cell cycle control, cell division, chromosom...",37 celsius,glucose(2),putrescine,WT,511145,M9,NH4Cl(1),KH2PO4(3) Na2HPO4(6.8),MgSO4(0.24),CaCl2(0.1)


## significance

In [4]:
# a bit behind annotations currently used in AVA MS.
# Still uses same approach to counted, just not all data-types have been consolidated into the regulators feat col


comp_mut_df = pd.DataFrame(columns=["observed mutation count"])
for i, r in exp_muts_df.iterrows():
    for f, links in r["regulator links"].items():
        if f not in comp_mut_df.index:
            df = pd.DataFrame({"observed mutation count": len(links)}, index=[f])
            comp_mut_df = comp_mut_df.append(df)
        else:
            comp_mut_df.loc[f, "observed mutation count"] += len(links)
            
comp_mut_df

,observed mutation count
YjjQ,14
Fur,15
MatA,21
LrhA,14
OmpR,16
RcsAB,14
QseB,14
H-NS,19
IHF,24
FliZ,14


In [5]:
for _, r in exp_muts_df.iterrows():
    for feat_d in r["regulators"]:
        feat_d["mutation set count"] = comp_mut_df.loc[feat_d["name"]]["observed mutation count"]

In [6]:
reg_len_df = pd.read_pickle("./data/regulon_len_df.pkl")
reg_len_df.head()

,regulon length
transcription factor,
SlyA,30460
AllS,3253
GlcC,9621
UxuR,16283
BolA,2498


In [7]:
import numpy as np


comp_mut_df["length"] = comp_mut_df.apply(lambda r: reg_len_df.loc[r.index, "regulon length"])
total_seq_len = np.sum(comp_mut_df["length"])
comp_mut_df["proportion"] = comp_mut_df["length"].apply(lambda comp_len: comp_len/total_seq_len)
comp_mut_df.head()

,observed mutation count,length,proportion
YjjQ,14,7684,0.002199
Fur,15,159408,0.045613
MatA,21,8018,0.002294
LrhA,14,7210,0.002063
OmpR,16,20270,0.005800


In [8]:
from collections import Counter

comp_mut_df['equal or larger count'] = 0
component_l = comp_mut_df.index.tolist()
proportion_l = comp_mut_df["proportion"].tolist()
num_muts = np.sum(comp_mut_df["observed mutation count"])

# slides from UW speak about the relationship of permutations and precision.
# should read through them to get a sense of what is the least iterations that should execute.

num_itr = 10000
for i in range(0, num_itr):
    random_mut_comp_a = np.random.choice(
        a=component_l,
        size=int(num_muts),
        p=proportion_l,
        replace=True,
    )
    random_mut_comp_count_d = Counter(random_mut_comp_a)
    for component in random_mut_comp_count_d.keys():
        if random_mut_comp_count_d[component] >= comp_mut_df.loc[component, 'observed mutation count']:
            comp_mut_df.loc[component, 'equal or larger count'] += 1

comp_mut_df["p value"] = comp_mut_df["equal or larger count"].apply(
    lambda count: count/num_itr)

comp_mut_df["significant"] = comp_mut_df["p value"].apply(lambda p_val: True if p_val < ENRICH_ALPHA else False)

display(comp_mut_df.head())

,observed mutation count,length,proportion,equal or larger count,p value,significant
YjjQ,14,7684,0.002199,0,0.0000,True
Fur,15,159408,0.045613,2985,0.2985,False
MatA,21,8018,0.002294,0,0.0000,True
LrhA,14,7210,0.002063,0,0.0000,True
OmpR,16,20270,0.005800,0,0.0000,True


In [9]:
from statsmodels.stats import multitest
pvals = comp_mut_df["p value"]
rejects, pvals_corrected, alphacSidak, alphacBonf = multitest.multipletests(
    pvals=pvals,
    alpha=ENRICH_ALPHA,
    method=MULTI_HYP_CORR_METHOD)

pd.options.display.max_rows = 500
comp_mut_df["corrected p value"] = pvals_corrected
comp_mut_df["corrected significance"] = rejects
comp_mut_df.head()

,observed mutation count,length,proportion,equal or larger count,p value,significant,corrected p value,corrected significance
YjjQ,14,7684,0.002199,0,0.0000,True,0.0,True
Fur,15,159408,0.045613,2985,0.2985,False,1.0,False
MatA,21,8018,0.002294,0,0.0000,True,0.0,True
LrhA,14,7210,0.002063,0,0.0000,True,0.0,True
OmpR,16,20270,0.005800,0,0.0000,True,0.0,True


In [10]:
# # not wanting to consider mutation events with only 1 features as being significant
comp_mut_df["final significance"] = comp_mut_df.apply(lambda r: r["corrected significance"] if r["observed mutation count"] > 1 else False, axis=1)
# comp_mut_df.head()

In [11]:
for i, r in exp_muts_df.iterrows():
    for d in r["regulators"]:
        if d["name"] in comp_mut_df.index:
            d["significantly enriched"] = comp_mut_df.loc[d["name"]]["final significance"]
#             d["significant"] = comp_mut_df.loc[d["name"]]["final significance"]

In [12]:
assert(len(exp_muts_df.exp.unique())==1)
exp = exp_muts_df.exp.unique()[0]
comp_mut_df.to_csv("./data/supp/permutation_test_results_per_experiment/"+exp+"_mut_reg_signif.csv")

In [13]:
exp_muts_df.to_pickle("./data/11_df.pkl")

In [14]:
# import seaborn as sns
# import matplotlib
# import matplotlib.pyplot as plt
# %matplotlib inline
# plt.rcParams["figure.dpi"] = 100
# sns.set_context("talk")
# sns.set_style("ticks")


# def _is_signif(name, regulons):
#     is_signif = False
#     for d in regulons:
#         if d["name"] == name:
#             is_signif = d["significant"]
#     return is_signif


# reg_cnt_d = dict()
# for _, r in exp_muts_df.iterrows():
#     reg_link_d = r["regulator links"]
#     for reg, hits in reg_link_d.items():
#         t = reg        
#         if _is_signif(reg, r["regulators"]):
#             t = '*' + t
#         if t not in reg_cnt_d.keys(): reg_cnt_d[t] = 0
#         reg_cnt_d[t] += len(hits)
# df = pd.DataFrame.from_dict(reg_cnt_d, orient='index', columns=["mutated feature count"])
# df = df.sort_values("mutated feature count")
# df = df[-20:]
# ax = df.plot.barh(y='mutated feature count',
# #                   figsize=(10,25),
#                   figsize=(5,10),
#                   color="#4C72B0",
#                   width=1
#                  )
# for i, v in enumerate(df["mutated feature count"]):
#     ax.text(v, i - 0.2, str(v))
# sns.despine(ax=ax, top=True, right=True)
# ax.set_title("Top 20 Regulon mutated feature count")
# ax.get_legend().remove()

In [15]:
# reg_cnt_d = dict()
# for _, r in exp_muts_df.iterrows():
#     reg_link_d = r["regulator links"]
#     for reg, hits in reg_link_d.items():
#         t = reg
#         if _is_signif(reg, r["regulators"]):
#             t = '*' + t
#             if t not in reg_cnt_d.keys(): reg_cnt_d[t] = 0
#             reg_cnt_d[t] += len(hits)

# if len(reg_cnt_d) > 0:
#     df = pd.DataFrame.from_dict(reg_cnt_d, orient='index', columns=["mutated feature count"])
#     df = df.sort_values("mutated feature count")
#     ax = df.plot.barh(y='mutated feature count',
#     #                   figsize=(10,25),
#                       figsize=(5,10),
#                       color="#4C72B0",
#                       width=1
#                      )
#     for i, v in enumerate(df["mutated feature count"]):
#         ax.text(v, i - 0.2, str(v))
#     sns.despine(ax=ax, top=True, right=True)
#     ax.set_title("Significant regulon mutated feature count")
#     ax.get_legend().remove()